In [41]:
import os
os.environ["DIAMBRAROMSPATH"] = os.getcwd() 
os.environ["DIAMBRA_ENVS"] =  " ".join([f"""127.0.0.1:{50051 + i}""" for i in range(1)])
import diambra
import wandb
from wandb.integration.sb3 import WandbCallback
from diambra.arena.stable_baselines3.make_sb3_env import make_sb3_env, EnvironmentSettings
from diambra.arena import EnvironmentSettings, WrappersSettings
from stable_baselines3 import DQN
from diambra.arena import SpaceTypes
from diambra.arena import Roles, SpaceTypes, load_settings_flat_dict
import yaml
import json
def build_env(params, train : bool = True, render_mode=None):
    params["settings"]["action_space"] = SpaceTypes.DISCRETE if params["settings"]["action_space"] == "discrete" else SpaceTypes.MULTI_DISCRETE
    settings = load_settings_flat_dict(EnvironmentSettings, params["settings"])
    # Wrappers Settings
    wrappers_settings = load_settings_flat_dict(WrappersSettings, params["wrappers_settings"])
    # Create environment
    env, _ = make_sb3_env("sfiii3n", settings, wrappers_settings, render_mode=render_mode, no_vec=True)
    return env

def load_config(cfg_file):
    # Read the cfg file
    yaml_file = open(cfg_file)
    params = yaml.load(yaml_file, Loader=yaml.FullLoader)
    print("Config parameters = ", json.dumps(params, sort_keys=True, indent=4))
    yaml_file.close()
    return params


In [42]:
cfgFile = "/home/chenningcong/Desktop/sfiii/agents/stable_baselines3/cfg_files/sfiii3n/sr6_128x4_das_nc.yaml"
env = build_env(load_config(cfgFile), True, render_mode=None)

INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:... done.


Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.94,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 6
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

INFO:diambra.arena.arena_gym:EnvironmentSettings(game_id='sfiii3n', frame_shape=(128, 128, 1), step_ratio=6, disable_keyboard=True, disable_joystick=True, render_mode=None, splash_screen=True, rank=0, env_address='127.0.0.1:50051', grpc_timeout=600, seed=1732531984, difficulty=6, continue_game=0.0, show_final=False, tower=3, _last_seed=1732531984, pb_model=game_id: "sfiii3n"
frame_shape {
  h: 128
  w: 128
  c: 1
}
step_ratio: 6
n_players: 1
disable_keyboard: true
disable_joystick: true
action_spaces: DISCRETE
episode_settings {
}
, n_players=1, action_space=1, role=None, characters='Ryu', outfits=2, super_art=None, fighting_style=None, ultimate_style=None, speed_mode=None)


In [43]:
from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

In [44]:
from stable_baselines3 import PPO
trainedModel = "/home/chenningcong/Desktop/sfiii/agents/stable_baselines3/results/sfiii3n/sr6_128x4_das_nc/model/model.zip"
agent = PPO.load(trainedModel)

/home/chenningcong/miniconda3/envs/myenv/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/chenningcong/miniconda3/envs/myenv/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/chenningcong/miniconda3/envs/myenv/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range_vf. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/chenningcong/miniconda3/envs/myenv/lib/python3.11/site-packages/stable_baselines3/common

In [47]:
env

<Monitor<FlattenFilterDictObs<RoleRelativeObservation<NormalizeObservation<ActionsStack<AddLastActionToObservation<FrameStack<NoAttackButtonsCombinations<NormalizeReward<DiambraGym1P instance>>>>>>>>>>

In [53]:
import cv2
from collections import OrderedDict
import numpy as np
import time
def test_agent_episode(env, agent, seed=42, deterministic : bool=False):   
    # Environment reset
    observation = env.reset()[0]
    # Agent-Environment interaction loop
    total_reward = 0
    while True:
        # (Optional) Environment rendering
        env.render()
        
        # Action random sampling
        actions, _state = agent.predict(observation, deterministic=False)
        # Environment stepping
        observation, reward, terminated, truncated, info = env.step(int(actions))
        total_reward += reward
        # Episode end (Done condition) check
        if terminated or truncated:
            observation = env.reset()
            break

    # # Environment shutdown
    # env.close()
    print(f"evaluating {total_reward}")
    # Return success
    return total_reward

def test_agent_episodic(env, agent, deterministic, seed=42, preprocess=preprocess, n_episode=1):
    rrs = [test_agent_episode(env, agent, seed, preprocess) for i in range(n_episode)]
    rrs = np.array(rrs)
    return np.mean(rrs), np.std(rrs)

test_agent_episodic(env, agent, deterministic=False, seed=42, n_episode=10)

evaluating -1.1677018633540373
evaluating -1.1055900621118009


KeyboardInterrupt: 

: 